In [9]:
import torch
%matplotlib inline
import IPython
from IPython.display import Image
import cv2 as cv
from confvision.models import *
from confvision.conformalizer import RiskConformalizer, Conformalizer

In [10]:
print(f"Setup complete. Using torch {torch.__version__}({torch.cuda.get_device_properties(0).name if torch.cuda.is_available()else 'CPU'})")

Setup complete. Using torch 2.0.0(NVIDIA GeForce RTX 4060 Laptop GPU)


# Train the YOLOV5
###  Use training set and validation set

In [11]:
#!python train.py  --img 640 --batch 16 --epochs 50 --data /project/conf_data.yaml --weights yolov5s.pt --freeze 10

# Apply to Calibration and Test set

#### Move Both prediction in the folder /Preds

In [12]:
#!python detect.py --weights runs/train/exp/weights/best.pt --img-size 640 --conf 0.4 --source /project/conformal_data/calibrate/images --save-conf --save-txt --hide-labels

In [13]:
#!python detect.py --weights runs/train/exp/weights/best.pt --img-size 640 --conf 0.4 --source /project/conformal_data/test/images  --save-conf  --save-txt

In [14]:
# Method : "additive" or "multiplicative"
method = "additive"
# Objectness threshold, between 0 and 1, default=0.3
objectness_threshold = 0.3
# IoU threshold, between 0 and 1, default=0.3
iou_threshold = 0.3
# Alpha, desired error rate, default=0.1
alpha = 0.1

In [15]:
model = YOLOv5()

#cal_preds, test_preds = model.load_predictions()

In [16]:
import cv2
import os

images_folder = 'preds/calibrate/images'
labels_folder = 'preds/calibrate/labels'
prediction_folder = 'preds/cal_preds'

images = []
true_boxes = []
scores_list = []
pred_boxes = []


for image_file in os.listdir(images_folder):
    # Check if the file is an image (jpg or png)
    if image_file.endswith('.jpg') or image_file.endswith('.png'):
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)
        images.append(image)

        # Create pred_box and scores
        txt_file = os.path.join(prediction_folder, os.path.splitext(image_file)[0] + '.txt')
        try:
            with open(txt_file, 'r') as file:
                bounding_boxes = []
                scores = []
                for line in file:
                    cls_id, x_center, y_center, width, height, score = map(float, line.strip().split())
                    bounding_boxes.append([cls_id, x_center, y_center, width, height])
                    scores.append(score)
                pred_boxes.append(bounding_boxes)
                scores_list.append(scores)
        except FileNotFoundError:
            # Create empty lists if the prediction file doesn't exist
            pred_boxes.append(np.inf)
            scores_list.append(np.inf)

        # Create true_box
        txt_file1 = os.path.join(labels_folder, os.path.splitext(image_file)[0] + '.txt')
        try:
            with open(txt_file1, 'r') as file:
                bounding_boxes = []
                for line in file:
                    cls_id, x_center, y_center, width, height = map(float, line.strip().split())
                    bounding_boxes.append([cls_id, x_center, y_center, width, height])
                true_boxes.append(bounding_boxes)
        except FileNotFoundError:
            # Create an empty list if the label file doesn't exist
            true_boxes.append([])


In [17]:
import cv2
import os

images_folder = 'preds/test/images'
labels_folder = 'preds/test/labels'
prediction_folder = 'preds/test_preds'

images_test = []
true_boxes_test = []
scores_list_test = []
pred_boxes_test = []


for image_file in os.listdir(images_folder):
    # Check if the file is an image (jpg or png)
    if image_file.endswith('.jpg') or image_file.endswith('.png'):
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)
        images_test.append(image)

        # Create pred_box and scores
        txt_file = os.path.join(prediction_folder, os.path.splitext(image_file)[0] + '.txt')
        try:
            with open(txt_file, 'r') as file:
                bounding_boxes = []
                scores = []
                for line in file:
                    cls_id, x_center, y_center, width, height, score = map(float, line.strip().split())
                    bounding_boxes.append([cls_id, x_center, y_center, width, height])
                    scores.append(score)
                pred_boxes_test.append(bounding_boxes)
                scores_list_test.append(scores)
        except FileNotFoundError:
            # Create empty lists if the prediction file doesn't exist
            pred_boxes_test.append(np.inf)
            scores_list_test.append(np.inf)

        # Create true_box
        txt_file1 = os.path.join(labels_folder, os.path.splitext(image_file)[0] + '.txt')
        try:
            with open(txt_file1, 'r') as file:
                bounding_boxes = []
                for line in file:
                    cls_id, x_center, y_center, width, height = map(float, line.strip().split())
                    bounding_boxes.append([cls_id, x_center, y_center, width, height])
                true_boxes_test.append(bounding_boxes)
        except FileNotFoundError:
            # Create an empty list if the label file doesn't exist
            true_boxes_test.append([])

In [18]:
cal_preds = Predictions(images, true_boxes, pred_boxes, scores_list)
test_preds = Predictions(images_test, true_boxes_test, pred_boxes_test, scores_list_test)

In [19]:
cal_preds.scores

[[0.479079, 0.499927, 0.616486, 0.642935, 0.792116],
 [0.602402],
 [0.625652],
 [0.570343],
 [0.461389, 0.630898, 0.711298, 0.802868, 0.881529],
 [0.423235, 0.456668, 0.460682, 0.560427],
 [0.453903],
 [0.755155],
 [0.689696, 0.745215],
 inf,
 [0.439137, 0.841157, 0.842925],
 [0.445095],
 [0.796763],
 [0.516161],
 [0.643422],
 [0.406748, 0.425717],
 [0.600537],
 [0.538514],
 [0.407642, 0.438213, 0.617486],
 inf,
 [0.75764],
 [0.727441],
 [0.403224],
 [0.405881, 0.673151, 0.696145, 0.833126, 0.911176],
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 [0.930161],
 inf,
 inf,
 inf,
 [0.807879, 0.900144],
 inf,
 [0.600074],
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 [0.799141, 0.874911],
 [0.599943, 0.773765],
 inf,
 inf,
 inf,
 inf,
 [0.538963, 0.847054],
 [0.594596, 0.710065, 0.901148],
 [0.858863, 0.880567],
 [0.559585, 0.913655],
 [0.848957, 0.884526],
 [0.788554, 0.807769, 0.854122],
 [0.819239, 

# Box-Wise Conformal Prediction
## Max Approach

In [20]:
conf_box_add_1c = Conformalizer(mode="box", method=method, coordinate_wise=False)
conf_box_add_1c.calibrate(cal_preds, objectness_threshold=objectness_threshold,
                          iou_threshold=iou_threshold, alpha=alpha)
conf_box_add_1c.conformalize(test_preds);
cal_preds.describe(objectness_threshold=objectness_threshold, iou_threshold=iou_threshold)
test_preds.describe(objectness_threshold=objectness_threshold, iou_threshold=iou_threshold)

Calibrating with alpha=0.1


  0%|                                                                                           | 0/250 [00:00<?, ?it/s]

[[0.0, 0.225962, 0.453125, 0.144231, 0.367788], [0.0, 0.475962, 0.3125, 0.149038, 0.451923], [1.0, 0.223558, 0.455529, 0.144231, 0.372596], [1.0, 0.409856, 0.564904, 0.439904, 0.745192], [0.0, 0.774038, 0.532452, 0.375, 0.915865]]


TypeError: list indices must be integers or slices, not str

## Bonferroni Correction
This approach uses a Bonferroni correct to compute a margin in each direction, possibly resulting in boxes that fit the ground truth better, but at a cost of being overly conservative

In [ ]:
conf_box_add_4c = Conformalizer(mode="box", method=method, coordinate_wise=True)
conf_box_add_4c.calibrate(cal_preds, objectness_threshold=objectness_threshold,
                          iou_threshold=iou_threshold, alpha=alpha)
conf_box_add_4c.conformalize(test_preds);
cal_preds.describe(objectness_threshold=objectness_threshold, iou_threshold=iou_threshold);
test_preds.describe(objectness_threshold=objectness_threshold, iou_threshold=iou_threshold);

# Image-wise Conformal Prediction and Conformal Risk Control
## Conformal Prediction with Hausdorff Distance
Intuitively, this method first proposed by De Grancey et al., aims to guarantee that, in expectation, 90% of images have at least 75% of their ground truth boxes included in their conformalized boxes.